In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import wandb
import time
import os
import copy
import splitfolders
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
from tqdm import tqdm

PyTorch Version:  1.10.1+cu102
Torchvision Version:  0.11.2+cu102


In [3]:
import splitfolders
import pathlib
from PIL import Image as PImage

In [ ]:
for root, _, fnames in sorted(os.walk('../data/dataset/biased_utensil_shape', followlinks=True)):
    print(root)
    for idx, fname in enumerate(sorted(fnames)):
        print(os.path.join(root, fname))
        temp = PImage.open(os.path.join(root, fname))
        temp = temp.convert('RGB')
        temp.save(os.path.join(root, fname),'jpeg')

In [2]:
splitfolders.ratio("../data/dataset/original_utensil", output="../data/dataset/original_utensil_train",
    seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

Copying files: 297 files [00:00, 386.69 files/s]


In [16]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "../data/dataset/original_utensil_train"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "inception"

# Number of classes in the dataset
num_classes = 16

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 30

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [19]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # for epoch in range(num_epochs):
    while(True):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            wandb.log({"loss": loss,"Accuracy":epoch_acc})
            if phase == 'train' and epoch_acc>=0.99:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                break
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [20]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

In [32]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [33]:
%env WANDB_PROJECT=finetune_inception_no_sample

env: WANDB_PROJECT=finetune_inception_no_sample


In [34]:
wandb.init()
wandb.watch(model_ft, log_freq=100)

[]

In [22]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [36]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias


In [37]:
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/29
----------
train Loss: 2.1105 Acc: 0.4211
val Loss: 0.8695 Acc: 0.6129

Epoch 1/29
----------
train Loss: 1.1977 Acc: 0.6391
val Loss: 0.6328 Acc: 0.9032

Epoch 2/29
----------
train Loss: 0.9960 Acc: 0.7293
val Loss: 0.5110 Acc: 0.8710

Epoch 3/29
----------
train Loss: 0.9601 Acc: 0.6955
val Loss: 0.4209 Acc: 0.9677

Epoch 4/29
----------
train Loss: 0.8568 Acc: 0.7556
val Loss: 0.4226 Acc: 0.9032

Epoch 5/29
----------
train Loss: 0.7278 Acc: 0.7669
val Loss: 0.3431 Acc: 0.9677

Epoch 6/29
----------
train Loss: 0.7957 Acc: 0.7594
val Loss: 0.3350 Acc: 0.9032

Epoch 7/29
----------
train Loss: 0.7040 Acc: 0.8008
val Loss: 0.3955 Acc: 0.9032

Epoch 8/29
----------
train Loss: 0.8296 Acc: 0.7406
val Loss: 0.3095 Acc: 0.9355

Epoch 9/29
----------
train Loss: 0.7385 Acc: 0.7857
val Loss: 0.3296 Acc: 0.9032

Epoch 10/29
----------
train Loss: 0.6876 Acc: 0.8045
val Loss: 0.3517 Acc: 0.9032

Epoch 11/29
----------
train Loss: 0.6401 Acc: 0.8195
val Loss: 0.3487 Acc: 0.9032

Ep

In [38]:
wandb.finish()

Accuracy,▁▃▇▅▅█▇▅▅▇▇▅▆▇▇▆▅▇▇▅▆▇█▆▆▇▇▆▆▇▇▆▆▇▇▅▆▇▇▇
loss,▅▄▃▄▃▂▂▃▆▂▂▇▃▂▁▂▂▂▁▇▃▁▁▇█▁▂▂▃▂▂▄▁▁▁▂▂▂▂▁
Accuracy,0.90323
loss,0.20572


In [ ]:
model_ft

In [23]:
image_datasets['train'].class_to_idx

{'DINNER_FORK_KNIFE': 0, 'FISH_SLICE': 1, 'TEA_SPOON': 2}

In [17]:
torch.save({
    'acc': ,
    'model':model_ft,
    'classes': image_datasets['train'].class_to_idx,
    'model_state_dict': model_ft.state_dict(),
    }, "inception_finetune_original.pkl")
